In [1]:
# Install necessary packages
!pip install python-binance pandas numpy plotly nbformat ipykernel scikit-learn seaborn matplotlib tensorflow

import pandas as pd
import numpy as np
from binance.client import Client
import plotly.graph_objects as go
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, precision_score
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

DEPRECATION: Loading egg at /Users/jcamilovasquezm/visualStudioProjects/TWS_IBKR_Trading/venv/lib/python3.11/site-packages/ibapi-10.34.2-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


KeyboardInterrupt: 

In [2]:
# In your notebook cell
from app_df import PTLClient, get_historical_dataframe, DEFAULT_HOST, PAPER_TRADING_PORT, DEFAULT_CLIENT_ID

# Create an instance of your client
client = PTLClient(DEFAULT_HOST, PAPER_TRADING_PORT, DEFAULT_CLIENT_ID)

# Now request historical data (for example, EUR USD data) and get it as a DataFrame.
# Adjust the parameters as needed (symbol, timeframe, etc.)
df = get_historical_dataframe(client, "EUR", "1 hour", "20241228 12:00:00 US/Eastern", "12 M")

print(df.head())

ERROR 1 1739894448812 504 Not connected


getting bar data for EUR 1 hour


KeyboardInterrupt: 

In [2]:
df.head()

,Open,High,Low,Close,Volume
Datetime,,,,,
2024-01-02 17:15:00-05:00,1.09400,1.09445,1.09396,1.09410,-1
2024-01-02 18:00:00-05:00,1.09410,1.09436,1.09409,1.09414,-1
2024-01-02 19:00:00-05:00,1.09414,1.09506,1.09412,1.09493,-1
2024-01-02 20:00:00-05:00,1.09493,1.09548,1.09448,1.09513,-1
2024-01-02 21:00:00-05:00,1.09513,1.09531,1.09486,1.09524,-1


In [3]:
df.shape

(6166, 5)

In [4]:
# -------------------------------
# Add Technical Indicators
# -------------------------------
def add_ema(df, periods=[20, 50, 100, 200]):
    for period in periods:
        df[f'EMA_{period}'] = df['Close'].ewm(span=period, adjust=False).mean()
    return df

df = add_ema(df)

In [5]:
# -------------------------------
# Prepare Data (Feature Engineering)
# -------------------------------
def prepare_data(data, sma_window=14, rsi_window=14):
    # Calculate basic indicators
    data['Returns'] = data['Close'].pct_change()
    data['SMA'] = data['Close'].rolling(window=sma_window).mean()
    data['Std'] = data['Close'].rolling(window=sma_window).std()
    data['Candle_Binary'] = (data['Returns'] > 0).astype(int)
    data['Momentum'] = data['Close'] - data['Close'].shift(5)
    data['Volatility'] = data['Returns'].rolling(window=5).std()
    data['ma5'] = data['Close'].rolling(window=5).mean()
    data['ma10'] = data['Close'].rolling(window=10).mean()
    
    # RSI calculation
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=rsi_window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=rsi_window).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))
    
    # Bar UpDown
    data['Up_Bar'] = (data['Close'] > data['Open']) & (data['Open'] > data['Close'].shift(1))
    data['Down_Bar'] = (data['Close'] < data['Open']) & (data['Open'] < data['Close'].shift(1))
    
    # Create features – you can choose a combination that works best
    features = ['Returns', 'Volatility', 'ma5', 'ma10', 'Momentum']
    X = data[features].shift(1)  # Shift by 1 to avoid look-ahead bias
    y = data['Candle_Binary']
    
    # Remove NaN values
    X = X.dropna()
    y = y[X.index]
    
    return X, y

In [9]:
X, y = prepare_data(df, sma_window=14)

In [10]:
# -------------------------------
# Create Sequences for LSTM
# -------------------------------
def create_sequences(X, y, window_size):
    Xs, ys = [], []
    for i in range(window_size, len(X)):
        Xs.append(X.iloc[i-window_size:i].values)
        ys.append(y.iloc[i])
    return np.array(Xs), np.array(ys)

# Choose a sequence length (e.g., 10 time steps)
window_size = 10
X_seq, y_seq = create_sequences(X, y, window_size)
print("LSTM Input Shape:", X_seq.shape)  # (samples, timesteps, features)

LSTM Input Shape: (6146, 10, 5)


In [11]:
# -------------------------------
# Train-Test Split (Chronological)
# -------------------------------
split_idx = int(0.8 * len(X_seq))
X_train, X_test = X_seq[:split_idx], X_seq[split_idx:]
y_train, y_test = y_seq[:split_idx], y_seq[split_idx:]

In [12]:
# -------------------------------
# Build the LSTM Model
# -------------------------------
num_features = X_train.shape[2]
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(window_size, num_features)),
    Dropout(0.2),
    LSTM(50),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

/Users/jcamilovasquezm/visualStudioProjects/TWS_IBKR_Trading/venv/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 10, 50)         │        11,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,451 (122.86 KB)

 Trainable params: 31,451 (122.86 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
# -------------------------------
# Train the LSTM Model
# -------------------------------
history = model.fit(X_train, y_train, epochs=20, batch_size=32,
                    validation_split=0.1, verbose=1)
                    

# -------------------------------
# 6. Evaluate the LSTM Model
# -------------------------------
# Predict probabilities and round to 0/1
y_pred_prob = model.predict(X_test)

from sklearn.metrics import precision_recall_curve

precision, recall, thresholds = precision_recall_curve(y_test, y_pred_prob)
f1_scores = np.where(
    (precision + recall) == 0,
    0,
    2 * (precision * recall) / (precision + recall)
)
optimal_threshold = thresholds[np.argmax(f1_scores)]
print(f"Optimal threshold: {optimal_threshold}")



y_pred = (y_pred_prob > optimal_threshold).astype(int).flatten()



Epoch 1/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5170 - loss: 0.6944 - val_accuracy: 0.5224 - val_loss: 0.6922
Epoch 2/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4972 - loss: 0.6958 - val_accuracy: 0.5224 - val_loss: 0.6922
Epoch 3/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5075 - loss: 0.6936 - val_accuracy: 0.5224 - val_loss: 0.6926
Epoch 4/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5096 - loss: 0.6931 - val_accuracy: 0.5224 - val_loss: 0.6931
Epoch 5/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4879 - loss: 0.6943 - val_accuracy: 0.5224 - val_loss: 0.6929
Epoch 6/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4820 - loss: 0.6939 - val_accuracy: 0.5224 - val_loss: 0.6930
Epoch 7/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5007 - loss: 0.6932 - val_accuracy: 0.5224 - val_loss: 0.6931
Epoch 8/20
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5112 - loss: 0.6931 - val_accuracy: 0.

In [14]:
# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")
print("\nMean Squared Error Report:")
print(mse)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

Test Accuracy: 0.48

Mean Squared Error Report:
0.5154471544715448

Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.00      0.00       634
           1       0.48      1.00      0.65       596

    accuracy                           0.48      1230
   macro avg       0.49      0.50      0.33      1230
weighted avg       0.49      0.48      0.32      1230


Confusion Matrix:
[[  1 633]
 [  1 595]]


In [89]:
# Example usage: fetch 1h candles for BTCUSDT
symbol = 'BTCUSDT'
interval = '1h'
df_test = get_historical_klines_range(symbol, interval, "2025-01-02", "2025-02-15")
print(df_test.head())

                         Open      High       Low     Close      Volume
Datetime                                                               
2025-01-02 00:00:00  94591.78  95381.70  94392.00  95128.59  1181.76573
2025-01-02 01:00:00  95128.59  95516.00  94699.69  94862.72  1143.36617
2025-01-02 02:00:00  94862.73  95234.37  94803.03  95234.36   455.65000
2025-01-02 03:00:00  95234.37  95299.24  94913.75  95010.33   290.28586
2025-01-02 04:00:00  95010.34  96000.00  94984.84  95708.07   925.61919


In [90]:
df_test = add_ema(df_test)

In [91]:
X, y = prepare_data(df_test, sma_window=14)

# Choose a sequence length (e.g., 10 time steps)
window_size = 10
X_seq, y_seq = create_sequences(X, y, window_size)
print("LSTM Input Shape:", X_seq.shape)  # (samples, timesteps, features)

LSTM Input Shape: (1037, 10, 5)


In [92]:
df_sim = get_historical_klines_range(symbol, interval, "2025-01-02", "2025-02-15")
df_sim = add_ema(df_sim)
print("Simulation Data:")
print(df_sim.head())

# Prepare simulation features and target
X_sim, y_sim = prepare_data(df_sim, sma_window=14)

# Create sequences for LSTM; note the window_size must match the one used during training
X_seq_sim, y_seq_sim = create_sequences(X_sim, y_sim, window_size)
print("LSTM Simulation Input Shape:", X_seq_sim.shape)

# Initialize simulation parameters
capital = 100000.0  # starting capital in USD
trades = []
long_trades = 0
short_trades = 0

# For simulation, we assume that each sequence's prediction corresponds to trading on the next candle.
# Because our sequences are created from X_sim (with indices aligned to df_sim),
# the candle to trade is the one at index: X_sim.index[window_size + i].
sim_indices = X_sim.index[window_size:]

for i in range(len(X_seq_sim)):
    # Get prediction from LSTM model for the current sequence
    seq = X_seq_sim[i:i+1]  # shape (1, window_size, features)
    pred_prob = model.predict(seq)[0, 0]
    #predicted_signal = int(np.round(pred_prob))  # 1 for bullish, 0 for bearish
    predicted_signal = (pred_prob > 0.51).astype(int).flatten()
    
    # Identify the corresponding candle for trading from the simulation DataFrame
    candle_idx = sim_indices[i]
    candle = df_sim.loc[candle_idx]
    
    # Simulate trade:
    # If bullish (1), take a long trade: profit = (Close - Open)
    # If bearish (0), take a short trade: profit = (Open - Close)
    if predicted_signal == 1:
        profit = candle['Close'] - candle['Open']
        long_trades += 1
    else:
        profit = candle['Open'] - candle['Close']
        short_trades += 1
    
    capital += profit
    trades.append({
        'date': candle_idx,
        'predicted_signal': predicted_signal,
        'Open': candle['Open'],
        'Close': candle['Close'],
        'profit': profit,
        'capital': capital
    })

Simulation Data:
                         Open      High       Low     Close      Volume  \
Datetime                                                                  
2025-01-02 00:00:00  94591.78  95381.70  94392.00  95128.59  1181.76573   
2025-01-02 01:00:00  95128.59  95516.00  94699.69  94862.72  1143.36617   
2025-01-02 02:00:00  94862.73  95234.37  94803.03  95234.36   455.65000   
2025-01-02 03:00:00  95234.37  95299.24  94913.75  95010.33   290.28586   
2025-01-02 04:00:00  95010.34  96000.00  94984.84  95708.07   925.61919   

                           EMA_20        EMA_50       EMA_100       EMA_200  
Datetime                                                                     
2025-01-02 00:00:00  95128.590000  95128.590000  95128.590000  95128.590000  
2025-01-02 01:00:00  95103.269048  95118.163725  95123.325248  95125.944527  
2025-01-02 02:00:00  95115.753900  95122.720442  95125.523955  95127.023288  
2025-01-02 03:00:00  95105.713529  95118.312974  95123.242887  9512

In [93]:
# Convert simulation results to a DataFrame and display
trades_df = pd.DataFrame(trades)
print("\nTrading Simulation Results (first 30 rows):")
print(trades_df.head(30))
print(f"\nTotal Long Trades: {long_trades}, Total Short Trades: {short_trades}")
print("\nFinal Capital: $", round(capital, 2))


Trading Simulation Results (first 30 rows):
                  date predicted_signal      Open     Close  profit    capital
0  2025-01-02 20:00:00              [0]  97317.74  97306.46   11.28  100011.28
1  2025-01-02 21:00:00              [0]  97306.46  97212.68   93.78  100105.06
2  2025-01-02 22:00:00              [0]  97212.69  96911.04  301.65  100406.71
3  2025-01-02 23:00:00              [0]  96911.04  96984.79  -73.75  100332.96
4  2025-01-03 00:00:00              [0]  96984.79  96825.82  158.97  100491.93
5  2025-01-03 01:00:00              [0]  96825.83  96985.47 -159.64  100332.29
6  2025-01-03 02:00:00              [0]  96985.47  96966.85   18.62  100350.91
7  2025-01-03 03:00:00              [0]  96966.85  97062.30  -95.45  100255.46
8  2025-01-03 04:00:00              [0]  97062.30  96813.41  248.89  100504.35
9  2025-01-03 05:00:00              [0]  96813.49  96765.34   48.15  100552.50
10 2025-01-03 06:00:00              [0]  96765.35  96539.49  225.86  100778.36
11 2025